In [1]:
# 导入 Flask 应用框架
from flask import Flask,request,jsonify
# 导入 Flask-CORS 扩展，用于处理跨域问题
from flask_cors import CORS
# 导入 pymysql 模块，用于连接 MySQL 数据库
import pymysql
# 导入 json 模块，用于处理 JSON 数据  
import json

app = Flask(__name__)

# 定义一个函数来执行 SQL 语句  
def excecuteSql(sql):
    flag = '1'
    try:
        # 使用 pymysql 连接 MySQL 数据库  
        db = pymysql.connect(host='127.0.0.1',user='root',passwd='12345678',port=3306, db='tangniaobing')
        print('连接成功！')
        # 创建一个游标对象，用于执行 SQL 语句  
        cursor = db.cursor()
        #执行sql语句
        cursor.execute(sql)
        # 提交事务  
        db.commit()
        flag = '0'
        print("数据修改成功")

    except Exception as result:
        print(result)
        # 如果出现异常，回滚事务  
        db.rollback()
        print("数据修改失败")

    finally:
        # 关闭数据库连接
        db.close()

    return flag



#导入预测可能使用的包
import numpy as np# 用于数值计算
import pandas as pd# 用于数据处理和分析的库 
#from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier# 使用决策树分类器  
from sklearn.neural_network import MLPClassifier# 使用多层感知机分类器  
from sklearn.neighbors import KNeighborsClassifier# 使用 K-近邻分类器  
from sklearn.linear_model import LogisticRegression# 使用逻辑回归分类器 
from sklearn.ensemble import RandomForestClassifier# 使用随机森林分类器   
from sklearn.model_selection import train_test_split# 数据集切分函数，用于划分训练集和测试集
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score# 用于模型评估的函数和指标  
from sklearn.preprocessing import LabelEncoder# 对分类标签进行编码的函数  
from xgboost import XGBClassifier# 使用 XGBoost 分类器，一种基于梯度提升决策树的机器学习算法
# 数据标准化
# 数据标准化函数，用于特征缩放处理，使特征具有相同的尺度或规模范围，有助于模型的训练和预测。这个函数用于对数据进行预处理，以便于模型训练和预测。
from sklearn.preprocessing import StandardScaler
#用于封装保存预测模型的包
import joblib



#预测系统路由
@app.route('/forcast',methods=['POST'])
def forcast():
    # 初始化特征的标准化器
    ss_x = StandardScaler()
    
    #导入csv数据（已经清洗过后并保存过的数据）
    df = pd.read_csv("C:/Users/lenovo/Desktop/pima2-diabetes.csv")
    #   x为可能导致糖尿病的因素
    x = df.drop('Outcome', axis=1)
    #   y为诊断结果，1为确诊，0为未确诊
    y = df['Outcome']
    #   x为除Outcome 列之外的所有数据
    X = df.drop('Outcome', axis=1)
    #X = ss_x.fit_transform(X)
    #划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=100 / 5643, random_state=0)
    #使用逻辑回归分类器
    lr = LogisticRegression(random_state=0, max_iter=1000)
    #训练模型
    lr.fit(X_train, y_train)
    #对测试集进行预测
    y_pred = lr.predict(X_test)
    print('逻辑回归准确率')
    print(confusion_matrix(y_test, y_pred)) # 混淆矩阵用于展示分类模型的实际结果和预测结果之间的对比
    print(classification_report(y_test, y_pred))# 分类报告提供了关于分类模型的详细信息，包括每个类别的精度、召回率和F1分数等 
    print(accuracy_score(y_test, y_pred))# 准确度得分是分类模型正确预测的样本数占总样本数的比例
    print('模型得分:{:.2f}'.format(lr.score(X_test, y_test)))
    y_ = np.array(y_test)
    print('逻辑回归预测结果：', lr.predict(X_test))
    print('真实结果:         ', y_)
    print('--------------------------------------------------------------------------------------')
    joblib.dump(lr, 'model3.pkl')#将该模型保存为一个名为model3的pkl文件


    #接收post请求数据
    data = request.get_json()
    print(data['年龄'])
    
    #将数据存放到一个数组中
    indices = [data['年龄'], data['*天门冬氨酸氨基转换酶'], data['*丙氨酸氨基转换酶'], data['*碱性磷酸酶'], data['*r-谷氨酰基转换酶'], data['*总蛋白'],
               data['白蛋白'], data['*球蛋白'],data['白球比例'],data['甘油三酯'],data['总胆固醇'],
              data['高密度脂蛋白胆固醇'],data['低密度脂蛋白胆固醇'],data['尿素'],data['肌酐'],data['尿酸'],
               data['白细胞计数'],data['红细胞计数'],data['血红蛋白'],data['红细胞压积'],data['红细胞平均体积'],
               data['红细胞平均血红蛋白量'],data['红细胞平均血红蛋白浓度'],data['红细胞体积分布宽度'],data['血小板计数'],
              data['血小板平均体积'],data['血小板体积分布宽度'],data['血小板比积'],data['中性粒细胞%'],data['淋巴细胞%'],
               data['单核细胞%'],data['嗜酸细胞%'],data['嗜碱细胞%'],data['血糖']]

    l = []
    #indices = np.array(indices).reshape(1, -1)
    for index in indices:
        l.append(index)
    empty_indices = {"年龄": l[0], "*天门冬氨酸氨基转换酶": l[1], 
                         "*丙氨酸氨基转换酶": l[2], "*碱性磷酸酶": l[3],
                         "*r-谷氨酰基转换酶": l[4], "*总蛋白": l[5], 
                         "白蛋白": l[6], "*球蛋白": l[7],
                         "白球比例":l[8],"甘油三酯":l[9],"总胆固醇":l[10],"高密度脂蛋白胆固醇":l[11],
                         "低密度脂蛋白胆固醇":l[12],"尿素":l[13],"肌酐":l[14],"尿酸":l[15],
                         "白细胞计数":l[16],"红细胞计数":l[17],"血红蛋白":l[18],"红细胞压积":l[19],
                         "红细胞平均体积":l[20],"红细胞平均血红蛋白量":l[21],"红细胞平均血红蛋白浓度":l[22],"红细胞体积分布宽度":l[23],
                         "血小板计数":l[24],"血小板平均体积":l[25],"血小板体积分布宽度":l[26],"血小板比积":l[27],
                         "中性粒细胞%":l[28],"淋巴细胞%":l[29],"单核细胞%":l[30],"嗜酸细胞%":l[31],
                         "嗜碱细胞%":l[32],"血糖":l[33]
                        }
    ## 创建一个空的DataFrame，名为da，并设置其索引为0  
    da = pd.DataFrame(empty_indices, index=[0])
    # 将DataFrame的索引设置为“年龄”列的值  
    da = da.set_index("年龄")
    # 将DataFrame保存为CSV文件，名为"new1.csv"  
    da.to_csv("new1.csv")
    # 从CSV文件"new1.csv"中读取数据，并创建一个新的DataFrame，名为df_1  
    df_1 = pd.read_csv("new1.csv")
    # 选取DataFrame的第一个数据行，并将其赋值给s
    s = df_1.iloc[0]
    # 将数据行s转换为numpy数组，并重塑为1行多列的形状  
    s = np.array(s).reshape(1, -1)
    # 从文件"model3.pkl"中加载一个已经训练好的分类器模型，并将其赋值给classifier  
    classifier = joblib.load('model3.pkl')
    # 使用分类器对数据行s进行预测，并将结果赋值给pre  
    pre = classifier.predict(s)
    # 获取预测结果pre的第一个值，并将其赋值给a  
    a = pre[0]
    # 根据预测结果a的值判断用户的健康状况
    if a ==1:
        res = '从您的数据来看，您有患糖尿病的风险'
    else:
        res = '您很健康，目前没有患糖尿病的风险'

    # 处理POST请求的逻辑
    # ...
 
    # 返回响应
    # 创建一个字典，包含预测结果res，并将其赋值给response_data  
    response_data =  {'预测结果': res}
    # 将response_data转换为json格式的响应，并返回该响应  
    return jsonify(response_data)

    
    
            



# 用户表Users
# 定义一个路由，用于插入用户数据  
@app.route('/Users/insert/<user_id>/<username>/<password>/<email>/<contact_number>')
def UsersInsert(user_id, username, password, email, contact_number):
    # 构造SQL插入语句，使用占位符避免SQL注入  
    sql = "insert into Users values({},'{}','{}','{}',{})".format(str(user_id), username, password, email,
                                                                  str(contact_number))
    print(sql)
    flag = excecuteSql(sql)
    return flag



# 定义一个路由，用于更新用户数据  
@app.route('/Users/update/<user_id>/<username>/<password>/<email>/<contact_number>')
def UsersUpdate(user_id, username, password, email, contact_number):
    sql = "update Users set username = '{}',password = '{}',email = '{}',contact_number={} where user_id = {}".format(
        username, password, email, str(contact_number), str(user_id))
    print(sql)
    flag = excecuteSql(sql)
    return flag

# 定义一个路由，用于删除用户数据  
@app.route('/Users/delete/<user_id>')
def UsersDelect(user_id):
    # 打印要删除的用户ID，方便调试  
    print(user_id)
    sql = "delete from Users where user_id =" + str(user_id)
    print(sql)
    flag = excecuteSql(sql)
    return flag

# 定义一个路由，用于选择所有用户数据  
@app.route('/Users/selectall')
def UsersSelectAll():
    # 初始化一个空列表，用于存储用户数据  
    UsersList = []
    try:
        # 连接MySQL数据库，参数分别为主机名、用户名、密码、端口和数据库名
        db = pymysql.connect(host='127.0.0.1',user='root',passwd='12345678',port=3306, db='tangniaobing')
        print('连接成功！')

        # 创建一个游标对象，用于执行SQL语句和获取查询结果  
        cursor = db.cursor()
        # 构造SQL查询语句，查询Users表中的所有数据
        sql = """select * FROM Users"""
        cursor.execute(sql)
        # 获取查询结果，结果为一个元组列表，每个元组包含一行数据  
        result = cursor.fetchall()
        # 遍历查询结果，将每行数据拆分为单独的字段，并存储到字典中，然后将字典添加到UsersList列表中  
        for x in result:
            user_id, username, password, email, contact_number = x[0], x[1], x[2], x[3], x[4]

            Users = {"user_id": str(user_id), "username": username,
                     "password": password, "email": email, "contact_number": str(contact_number)}
            UsersList.append(Users)

    except Exception as result:

        print("数据查询失败")
        print(result)

    finally:
        db.close()
        
    # 将UsersList列表转换为JSON格式的字符串，并打印出来，然后返回该字符串作为响应结果
    json_date = json.dumps(UsersList)
    print(json_date)

    return json_date


# ------------------------------------------------------------------------------------------
# 患者信息表 (PatientInfo)
# 定义一个路由，用于插入患者信息  
@app.route('/PatientInfo/insert/<patient_id>/<user_id>/<full_name>/<date_of_birth>/<gender>')
def PatientInfoInsert(patient_id, user_id, full_name, date_of_birth, gender):
    sql = "insert into PatientInfo values('{}','{}','{}',{},'{}')".format(str(patient_id), str(user_id), full_name,
                                                                          str(date_of_birth), gender)
    print(sql)
    flag = excecuteSql(sql)
    return flag

# 定义一个路由，用于更新患者信息  
@app.route('/PatientInfo/update/<patient_id>/<user_id>/<full_name>/<date_of_birth>/<gender>')
def PatientInfoUpdate(patient_id, user_id, full_name, date_of_birth, gender):
    sql = "update PatientInfo set user_id = '{}',full_name = '{}',date_of_birth = '{}',gender={} where patient_id = {}".format(
        str(user_id), full_name, str(date_of_birth), gender, str(patient_id))
    print(sql)
    flag = excecuteSql(sql)
    return flag

# 定义一个路由，用于删除患者信息 
@app.route('/PatientInfo/delete/<patient_id>')
def PatientInfoDelect(patient_id):
    print(patient_id)
    sql = "delete from PatientInfo where patient_id =" + str(patient_id)
    print(sql)
    flag = excecuteSql(sql)
    return flag

#定义一个路由用于查询所有患者信息
@app.route('/PatientInfo/selectall')
def PatientInfoSelectAll():
    # 初始化一个空列表，用于存储查询到的患者信息  
    PatientInfoList = []
    try:
        # 连接MySQL数据库，参数依次为主机地址、用户名、密码、端口和数据库名称 
        db = pymysql.connect(host='127.0.0.1',user='root',passwd='12345678',port=3306, db='tangniaobing')
        print('连接成功！')

        cursor = db.cursor()
        # 构造SQL查询语句，用于查询PatientInfo表中的所有数据  
        sql = """select * FROM PatientInfo"""
        cursor.execute(sql)
        # 获取查询结果，返回一个包含所有行的元组列表  
        result = cursor.fetchall()
         # 遍历查询结果，将每一行的数据提取出来并存储到字典中，然后添加到列表中  
        for x in result:
            patient_id, user_id, full_name, date_of_birth, gender = x[0], x[1], x[2], x[3], x[4]

            PatientInfo = {"patient_id": str(patient_id), "user_id": str(user_id),
                           "full_name": str(full_name), "date_of_birth": str(date_of_birth),
                           "gender": gender}
            PatientInfoList.append(PatientInfo)

    except Exception as result:

        print("数据查询失败")
        print(result)

    finally:
        db.close()

    
    # 将患者信息列表转换为JSON格式的字符串，并打印出来  
    json_date = json.dumps(PatientInfoList)
    print(json_date)
    # 返回JSON格式的数据，以便前端可以接收并显示数据  
    return json_date


# -----------------------------------------------------------------------
# 医生信息表 (DoctorInfo)
@app.route('/DoctorInfo/insert/<doctor_id>/<user_id>/<full_name>/<specialization>')
def DoctorInfoInsert(doctor_id, user_id, full_name, specialization):
    sql = "insert into DoctorInfo values({},{},'{}','{}')".format(str(doctor_id), str(user_id), full_name,
                                                                  specialization)
    print(sql)
    flag = excecuteSql(sql)
    return flag


@app.route('/DoctorInfo/update/<doctor_id>/<user_id>/<full_name>/<specialization>')
def DoctorInfoUpdate(doctor_id, user_id, full_name, specialization):
    sql = "update DoctorInfo set user_id = {},full_name = '{}',specialization = '{}' where doctor_id= {}".format(
        str(user_id), full_name, specialization, str(doctor_id))
    print(sql)
    flag = excecuteSql(sql)
    return flag


@app.route('/DoctorInfo/delete/<doctor_id>')
def DoctorInfoDelect(doctor_id):
    print(doctor_id)
    sql = "delete from DoctorInfo where doctor_id =" + str(doctor_id)
    print(sql)
    flag = excecuteSql(sql)
    return flag


@app.route('/DoctorInfo/selectall')
def DoctorInfoSelectAll():
    DoctorInfoList = []
    try:
        db = pymysql.connect(host='127.0.0.1',user='root',passwd='12345678',port=3306, db='tangniaobing')
        print('连接成功！')

        cursor = db.cursor()
        sql = """select * FROM DoctorInfo"""
        cursor.execute(sql)
        result = cursor.fetchall()
        for x in result:
            doctor_id, user_id, full_name, specialization = x[0], x[1], x[2], x[3]

            DoctorInfo = {"doctor_id": str(doctor_id), "user_id": str(user_id),
                          "full_name": full_name,
                          "specialization": specialization}
            DoctorInfoList.append(DoctorInfo)

    except Exception as result:

        print("数据查询失败")
        print(result)

    finally:
        db.close()

    json_date = json.dumps(DoctorInfoList)
    print(json_date)

    return json_date


# -------------------------------------------------------------------
# 预约信息表 (Appointments)
@app.route('/Appointments/insert/<appointment_id>/<patient_id>/<doctor_id>/<appointment_date>/<appointment_time>')
def AppointmentsInsert(appointment_id, patient_id, doctor_id, appointment_date, appointment_time):
    sql = "insert into Appointments values({},{},{},{},{})".format(str(appointment_id), str(patient_id), str(doctor_id),
                                                                   str(appointment_date), str(appointment_time))
    print(sql)
    flag = excecuteSql(sql)
    return flag


@app.route('/Appointments/update/<appointment_id>/<patient_id>/<doctor_id>/<appointment_date>/<appointment_time>')
def AppointmentsUpdate(appointment_id, patient_id, doctor_id, appointment_date, appointment_time):
    sql = "update Appointments set patient_id = {},doctor_id = {},appointment_date = {},appointment_time={} where appointment_id = {}".format(
        str(patient_id), str(doctor_id), str(appointment_date), str(appointment_time), str(appointment_id))
    print(sql)
    flag = excecuteSql(sql)
    return flag


@app.route('/Appointments/delete/<appointment_id>')
def AppointmentsDelect(appointment_id):
    print(appointment_id)
    sql = "delete from Appointments where appointment_id =" + str(appointment_id)
    print(sql)
    flag = excecuteSql(sql)
    return flag


@app.route('/Appointments/selectall')
def AppointmentsSelectAll():
    AppointmentsList = []
    try:
        db = pymysql.connect(host='127.0.0.1',user='root',passwd='12345678',port=3306, db='tangniaobing')
        print('连接成功！')

        cursor = db.cursor()
        sql = """select * FROM Appointments"""
        cursor.execute(sql)
        result = cursor.fetchall()
        for x in result:
            appointment_id, patient_id, doctor_id, appointment_date, appointment_time = x[0], x[1], x[2], x[3], x[4]

            Appointments = {"appointment_id": str(appointment_id), "patient_id": str(patient_id),
                            "doctor_id": str(doctor_id), "appointment_date": str(appointment_date),
                            "appointment_time": str(appointment_time)}
            AppointmentsList.append(Appointments)

    except Exception as result:

        print("数据查询失败")
        print(result)

    finally:
        db.close()

    json_date = json.dumps(AppointmentsList)
    print(json_date)

    return json_date


# -------------------------------------------------------------------------------
# 诊断信息表 (Diagnosis)
@app.route(
    '/Diagnosis/insert/<diagnosis_id>/<patient_id>/<doctor_id>/<appointment_id>/<blood_id>/<liver_id>/<renin_id>')
def DiagnosisInsert(diagnosis_id, patient_id, doctor_id, appointment_id, blood_id, liver_id, renin_id):
    sql = "insert into Diagnosis values({},{},{},{},{},{},{})".format(str(diagnosis_id), str(patient_id),
                                                                      str(doctor_id), str(appointment_id),
                                                                      str(blood_id), str(liver_id), str(renin_id))
    print(sql)
    flag = excecuteSql(sql)
    return flag


@app.route(
    '/Diagnosis/update/<diagnosis_id>/<patient_id>/<doctor_id>/<appointment_id>/<blood_id>/<liver_id>/<renin_id>')
def DiagnosisUpdate(diagnosis_id, patient_id, doctor_id, appointment_id, blood_id, liver_id, renin_id):
    sql = "update Diagnosis set patient_id = {},doctor_id = {},appointment_id = {},blood_id = {},Liver_id = {},Renin_id={} where diagnosis_id = {}".format(
        str(patient_id), str(doctor_id), str(appointment_id), str(blood_id), str(liver_id), str(renin_id),
        str(diagnosis_id))
    print(sql)
    flag = excecuteSql(sql)
    return flag


@app.route('/Diagnosis/delete/<diagnosis_id>')
def DiagnosisDelect(diagnosis_id):
    print(diagnosis_id)
    sql = "delete from Diagnosis where diagnosis_id =" + str(diagnosis_id)
    print(sql)
    flag = excecuteSql(sql)
    return flag


@app.route('/Diagnosis/selectall')
def DiagnosisSelectAll():
    DiagnosisList = []
    try:
        db = pymysql.connect(host='127.0.0.1',user='root',passwd='12345678',port=3306, db='tangniaobing')
        print('连接成功！')

        cursor = db.cursor()
        sql = """select * FROM Diagnosis"""
        cursor.execute(sql)
        result = cursor.fetchall()
        for x in result:
            diagnosis_id, patient_id, doctor_id, appointment_id, blood_id, liver_id, renin_id = x[0], x[1], x[2], \
                x[3], x[4], x[5], x[6]

            Diagnosis = {"diagnosis_id": str(diagnosis_id), "patient_id": str(patient_id), "doctor_id": str(doctor_id),
                         "appointment_id": str(appointment_id), "blood_id": str(blood_id),
                         "liver_id": str(liver_id), "renin_id": str(renin_id)}
            DiagnosisList.append(Diagnosis)

    except Exception as result:

        print("数据查询失败")
        print(result)

    finally:
        db.close()

    json_date = json.dumps(DiagnosisList)
    print(json_date)

    return json_date


# -----------------------------------------------------------------------------------------
# 血常规检测表 (BloodRoutine)
@app.route('/Bloodroutine/insert/<blood_id>/<patient_id>/<doctor_id>/<appointment_id>/<Neu>/<Lym>/<Mon>/<Eos>/<Bas>')
def BloodRoutineInsert(blood_id, patient_id, Neu, Lym, Mon, Eos, Bas, doctor_id, appointment_id):
    sql = "insert into BloodRoutine values({},{},{},{},{},{},{},{},{})".format(str(blood_id), str(patient_id),
                                                                               str(doctor_id), str(appointment_id),
                                                                               str(Neu), str(Lym), str(Mon),
                                                                               str(Eos), str(Bas))

    print(sql)
    flag = excecuteSql(sql)
    return flag


@app.route('/Bloodroutine/update/<blood_id>/<patient_id>/<doctor_id>/<appointment_id>/<Neu>/<Lym>/<Mon>/<Eos>/<Bas>')
def BloodRoutineUpdate(blood_id, patient_id, doctor_id, appointment_id, Neu, Lym, Mon, Eos, Bas):
    sql = "update BloodRoutine set patient_id = {},doctor_id = {},appointment_id={},Neu = {},Lym = {},Mon={},Eos = {},Bas = {} where blood_id = {}".format(
        str(patient_id), str(doctor_id), str(appointment_id), str(Neu), str(Lym), str(Mon), str(Eos), str(Bas),
        str(blood_id))
    print(sql)
    flag = excecuteSql(sql)
    return flag


@app.route('/Bloodroutine/delete/<blood_id>')
def BloodRoutineDelect(blood_id):
    print(blood_id)
    sql = "delete from BloodRoutine where blood_id =" + str(blood_id)
    print(sql)
    flag = excecuteSql(sql)
    return flag


@app.route('/Bloodroutine/selectall')
def BloodRoutineSelectAll():
    BloodRoutineList = []
    try:
        db = pymysql.connect(host='127.0.0.1',user='root',passwd='12345678',port=3306, db='tangniaobing')
        print('连接成功！')

        cursor = db.cursor()
        sql = """select * FROM BloodRoutine"""
        cursor.execute(sql)
        result = cursor.fetchall()
        for x in result:
            blood_id, patient_id, doctor_id, appointment_id, Neu, Lym, Mon, Eos, Bas = x[0], x[1], x[2], x[3], x[4], x[
                5], x[6], x[7], x[8]

            BloodRoutine = {"blood_id": str(blood_id), "patient_id": str(patient_id), "doctor_id": str(doctor_id),
                            "appointment_id": str(appointment_id), "Neu": str(Neu), "Lym": str(Lym), "Mon": str(Mon),
                            "Eos": str(Eos), "Bas": str(Bas)}
            BloodRoutineList.append(BloodRoutine)

    except Exception as result:

        print("数据查询失败")
        print(result)

    finally:
        db.close()

    json_date = json.dumps(BloodRoutineList)
    print(json_date)

    return json_date


# --------------------------------------------------------------------------
# 肝功能（Liver）
@app.route('/Liver/insert/<liver_id>/<patient_id>/<doctor_id>/<appointment_id>/<asp>/<alanine>/<alkaline>/<r_glutamyl>')
def LiverInsert(liver_id, patient_id, doctor_id, appointment_id, asp, alanine, alkaline, r_glutamyl):
    sql = "insert into Liver values({},{},{},{},{},{},{},{})".format(str(liver_id), str(patient_id), str(doctor_id),
                                                                     str(appointment_id), str(asp), str(alanine),
                                                                     str(alkaline), str(r_glutamyl))
    print(sql)
    flag = excecuteSql(sql)
    return flag


@app.route('/Liver/update/<liver_id>/<patient_id>/<doctor_id>/<appointment_id>/<asp>/<alanine>/<alkaline>/<r_glutamyl>')
def LiverUpdate(liver_id, patient_id, doctor_id, appointment_id, asp, alanine, alkaline, r_glutamyl):
    sql = "update Liver set asp = {},alanine = {},alkaline = {},r_glutamyl={},doctor_id = {},appointment_id={},patient_id={} where liver_id = {}".format(
        str(asp), str(alanine), str(alkaline), str(r_glutamyl), str(doctor_id), str(appointment_id), str(patient_id),
        str(liver_id))
    print(sql)
    flag = excecuteSql(sql)
    return flag


@app.route('/Liver/delete/<liver_id>')
def LiverDelect(liver_id):
    print(liver_id)
    sql = "delete from Liver where liver_id =" + str(liver_id)
    print(sql)
    flag = excecuteSql(sql)
    return flag


@app.route('/Liver/selectall')
def LiverSelectAll():
    LiverList = []
    try:
        db = pymysql.connect(host='127.0.0.1',user='root',passwd='12345678',port=3306, db='tangniaobing')
        print('连接成功！')

        cursor = db.cursor()
        sql = """select * FROM Liver"""
        cursor.execute(sql)
        result = cursor.fetchall()
        for x in result:
            liver_id, patient_id, doctor_id, appointment_id, asp, alanine, alkaline, r_glutamyl = x[0], x[1], x[2], x[
                3], x[4], x[5], x[6], x[7]

            Liver = {"liver_id": str(liver_id), "patient_id": str(patient_id), "doctor_id": str(doctor_id),
                     "appointment_id": str(appointment_id), "asp": str(asp), "alanine": str(alanine),
                     "alkaline": str(alkaline), "r_glutamyl": str(r_glutamyl)
                     }
            LiverList.append(Liver)

    except Exception as result:

        print("数据查询失败")
        print(result)

    finally:
        db.close()

    json_date = json.dumps(LiverList)
    print(json_date)

    return json_date


# ----------------------------------------------------------------------------
# 肾功能表（Renin）
@app.route('/Renin/insert/<renin_id>/<patient_id>/<doctor_id>/<appointment_id>/<urea>/<creatinine>/<uric_acid>')
def ReninInsert(renin_id, patient_id, doctor_id, appointment_id, urea, creatinine, uric_acid):
    sql = "insert into Renin values({},{},{},{},{},{},{})".format(str(renin_id), str(patient_id), str(doctor_id),
                                                                  str(appointment_id), str(urea), str(creatinine),
                                                                  str(uric_acid)
                                                                  )
    print(sql)
    flag = excecuteSql(sql)
    return flag


@app.route('/Renin/update/<renin_id>/<patient_id>/<doctor_id>/<appointment_id>/<urea>/<creatinine>/<uric_acid>')
def ReninUpdate(renin_id, patient_id, doctor_id, appointment_id, urea, creatinine, uric_acid):
    sql = "update Renin set patient_id={},doctor_id = {},appointment_id={},urea = {},creatinine = {},uric_acid = {} where renin_id = {}".format(
        str(patient_id), str(doctor_id), str(appointment_id), str(urea), str(creatinine), str(uric_acid), str(renin_id))
    print(sql)
    flag = excecuteSql(sql)
    return flag


@app.route('/Renin/delete/<renin_id>')
def ReninDelect(renin_id):
    print(renin_id)
    sql = "delete from Renin where renin_id =" + str(renin_id)
    print(sql)
    flag = excecuteSql(sql)
    return flag


@app.route('/Renin/selectall')
def ReninSelectAll():
    ReninList = []
    try:
        db = pymysql.connect(host='127.0.0.1',user='root',passwd='12345678',port=3306, db='tangniaobing')
        print('连接成功！')

        cursor = db.cursor()
        sql = """select * FROM Renin"""
        cursor.execute(sql)
        result = cursor.fetchall()
        for x in result:
            renin_id, patient_id, doctor_id, appointment_id, urea, creatinine, uric_acid = x[0], x[1], x[2], x[3], x[4], \
                x[5], x[6]

            Renin = {"renin_id": str(renin_id), "patient_id": str(patient_id), "doctor_id": str(doctor_id),
                     "appointment_id": str(appointment_id), "urea": str(urea), "creatinine": str(creatinine),
                     "uric_acid": str(uric_acid)
                     }
            ReninList.append(Renin)

    except Exception as result:

        print("数据查询失败")
        print(result)

    finally:
        db.close()

    json_date = json.dumps(ReninList)
    print(json_date)

    return json_date


@app.route('/result/selectall/<patient_id>')
def ResultSelectAll(patient_id):
    ResultList = []
    try:
        db = pymysql.connect(host='127.0.0.1',user='root',passwd='12345678',port=3306, db='tangniaobing')
        print('连接成功！')

        cursor = db.cursor()
        sql = """select dia.diagnosis_id,pat.patient_id,pat.full_name,pat.date_of_birth,pat.gender,app.appointment_date,app.appointment_time,doc.full_name  from diagnosis dia,patientinfo pat,appointments app,doctorinfo doc
 where dia.patient_id= pat.patient_id and dia.appointment_id = app.appointment_id and dia.doctor_id=doc.doctor_id
and pat.patient_id = '{}'""".format(patient_id)
        cursor.execute(sql)
        result = cursor.fetchall()
        for x in result:
            diagnosis_id, patient_id, full_name, date_of_birth, gender, appointment_date, appointment_time, docname = \
                x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7]

            Result = {"diagnosis_id": str(diagnosis_id), "patient_id": str(patient_id), "full_name": str(full_name),
                      "date_of_birth": str(date_of_birth), "gender": str(gender),
                      "appointment_date": str(appointment_date),
                      "appointment_time": str(appointment_time), "docname": str(docname)
                      }
            ResultList.append(Result)

    except Exception as result:

        print("数据查询失败")
        print(result)

    finally:
        db.close()

    json_date = json.dumps(ResultList)
    print(json_date)

    return json_date


@app.route('/result/select/<diagnosis_id>')
def ResultSelect(diagnosis_id):
    ResultList = []
    try:
        db = pymysql.connect(host='127.0.0.1',user='root',passwd='12345678',port=3306, db='tangniaobing')
        print('连接成功！')

        cursor = db.cursor()
        sql = """select blo.Neu,blo.Lym,blo.Mon,blo.Eos,blo.Bas,liver.asp,liver.alanine,liver.alkaline,
        liver.r_glutamyl,renin.urea,renin.creatinine,renin.uric_acid  from bloodroutine blo,liver,renin,diagnosis dia 
        where dia.renin_id = renin.renin_id and dia.blood_id = blo.blood_id and dia.liver_id = liver.liver_id and 
        dia.diagnosis_id = '{}'""".format(diagnosis_id)
        cursor.execute(sql)
        result = cursor.fetchall()
        for x in result:
            Neu, Lym, Mon, Eos, Bas, asp, alanine, alkaline, r_glutamyl, urea, creatinine, uric_acid = \
                x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11]

            Result = {"Neu": str(Neu), "Lym": str(Lym), "Mon": str(Mon), "Eos": str(Eos), "Bas": str(Bas),
                      "asp": str(asp), "alanine": str(alanine), "alkaline": str(alkaline), "r_glutamyl": str(r_glutamyl),
                      "urea": str(urea), "creatinine": str(creatinine), "uric_acid": str(uric_acid)
                      }
            ResultList.append(Result)

    except Exception as result:

        print("数据查询失败")
        print(result)

    finally:
        db.close()

    json_date = json.dumps(ResultList)
    print(json_date)

    return json_date


@app.route('/')
def hello():
    return "hello Flask"


CORS(app, resources={r'/*': {'origins': '*'}})
# set CORS for all routes
if __name__ == '__main__':
    app.run(host='127.0.0.1', port=1116)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:1116
Press CTRL+C to quit
127.0.0.1 - - [30/Dec/2023 01:00:01] "OPTIONS /forcast HTTP/1.1" 200 -
D:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\Anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
127.0.0.1 - - [30/Dec/2023 01:00:02] "POST /forcast HTTP/1.1" 200 -


逻辑回归准确率
[[71  1]
 [ 8 20]]
              precision    recall  f1-score   support

           0       0.90      0.99      0.94        72
           1       0.95      0.71      0.82        28

    accuracy                           0.91       100
   macro avg       0.93      0.85      0.88       100
weighted avg       0.91      0.91      0.91       100

0.91
模型得分:0.91
逻辑回归预测结果： [0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0
 0 0 0 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0]
真实结果:          [0 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1 1 1 0 0 0 0 1 1 0 0
 0 0 0 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1
 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0]
--------------------------------------------------------------------------------------
1


D:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\Anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
127.0.0.1 - - [30/Dec/2023 01:00:06] "POST /forcast HTTP/1.1" 200 -


逻辑回归准确率
[[71  1]
 [ 8 20]]
              precision    recall  f1-score   support

           0       0.90      0.99      0.94        72
           1       0.95      0.71      0.82        28

    accuracy                           0.91       100
   macro avg       0.93      0.85      0.88       100
weighted avg       0.91      0.91      0.91       100

0.91
模型得分:0.91
逻辑回归预测结果： [0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0
 0 0 0 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0]
真实结果:          [0 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1 1 1 0 0 0 0 1 1 0 0
 0 0 0 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1
 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0]
--------------------------------------------------------------------------------------
1
